In [4]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [5]:
df = pd.read_csv("../data/tox21.csv")
df.head()

NameError: name 'pd' is not defined

In [6]:
from rdkit import Chem
from rdkit.Chem import Atom, Mol, Bond
import psi4

mole = df["smiles"][0]
print(mole)
m = Chem.MolFromSmiles(mole)

atom = m.GetAtoms()[0]

atom.GetHybridization()

def atom_to_tensor(atom: Atom):
    # features list
    atomic_num = atom.GetAtomicNum()
    hybridization = atom.GetHybridization()
    valence = atom.GetTotalValence()

def bond_to_tensor(bond: Bond):
    # features list
    btype = bond.GetBondType()
    idx1, idx2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
    atom1_coords = m.GetConformer().GetAtomPosition(idx1)
    atom2_coords = m.GetConformer().GetAtomPosition(idx2)

    # convert to psi4 molecule for quantum estimations
    psi_bond = psi4.geometry(
        f"""
        2
        Atom1 {atom1_coords.x} {atom1_coords.y} {atom1_coords.z}
        Atom2 {atom2_coords.x} {atom2_coords.y} {atom2_coords.z}
        """
    )

    # setting bases for computing hartree fock energy from fock matrix
    psi4.set_options({ 'reference': 'rhf' })
    psi4.set_options({ 'basis': 'co-pvdz' })

    bond_length = psi_bond.bond_length(0, 1)
    energy, wavefunction = psi4.energy("scf/cc-pvdz")
    bond_order = wavefunction.bond_order(0, 1)
    alpha, beta = wavefunction.nalpha(), wavefunction.nbeta()
    oe_a, oe_b = wavefunction.epsilon_a_subset("A0"), wavefunction.epsilon_b_subset("AO")
    voe_a, voe_b = wavefunction.epsilon_a_subset("A0", "VIRTUAL"), wavefunction.epsilon_b_subset("AO", "VIRTUAL")
    nmr = psi4.nmr_shielding(psi_bond, atoms=[0,1])
    
# USE PSI4 TO GET LOCAL REGION INFORMATION

print(atom.GetNeighbors()[0].GetAtomicNum())
print(atom.GetBonds()[0])

ModuleNotFoundError: No module named 'rdkit'

In [ ]:
from pytorch

In [18]:
chunksize = 100

chunks = []

# for chunk in pd.read_csv("../data/tox21/tox21_dense_train.csv"):
#     chunks.append(chunk)

# data = chunks[0].iterrows()

df = pd.read_csv("../data/tox21/tox21_dense_train.csv")

In [19]:
df

,Unnamed: 0,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
0,NCGC00178831-03,5.436720e+07,13.053,2.176,3.194,23.112,15.868,1.496,15.127,12.592,...,2687.469,9241.018,115.371,-915.496,-39.983,290.078,2301.941,59.492,88.147,3.708
1,NCGC00166114-03,1.268818e+07,22.123,2.065,3.137,21.033,13.718,1.937,13.187,11.951,...,2184.384,3234.199,194.740,-1029.609,-34.205,235.360,1244.323,82.906,134.852,4.131
2,NCGC00263563-01,3.076932e+06,13.085,2.154,3.207,46.896,29.958,3.806,30.105,25.569,...,13803.524,76582.899,238.004,-4358.946,-106.537,868.685,15909.444,135.335,216.852,5.075
3,NCGC00013058-02,7.168569e+07,12.832,2.029,3.380,51.086,32.045,1.806,29.090,21.603,...,13807.345,50498.175,226.312,-2785.555,-61.923,763.288,9394.859,125.509,238.265,4.640
4,NCGC00167516-01,7.989702e+06,12.936,2.124,3.573,70.295,46.402,3.604,42.132,32.570,...,43231.286,163659.229,850.869,-21136.699,-367.122,1798.703,44681.209,362.168,317.901,7.845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12055,NCGC00261292-01,1.428572e+07,14.255,2.000,2.628,9.259,6.309,0.157,5.468,4.484,...,306.364,1435.590,76.419,-190.192,-13.757,100.624,250.402,19.275,29.148,2.581
12056,NCGC00261245-01,1.193182e+07,13.674,2.061,2.920,21.142,15.382,1.201,12.713,10.576,...,2528.642,12293.627,94.878,-595.491,-22.275,324.131,2034.439,49.446,93.636,3.666
12057,NCGC00260828-01,1.081800e+01,12.374,2.045,3.128,33.242,20.457,0.806,19.711,14.799,...,9171.300,44070.070,267.400,-2656.568,-104.039,874.679,8689.849,144.294,91.670,8.054
12058,NCGC00260687-01,3.229000e+00,12.543,2.267,2.700,10.251,7.381,0.587,6.455,5.857,...,391.790,1815.417,39.578,-105.234,-9.967,146.565,389.732,23.879,28.201,2.954


In [17]:
data[0][1].to_frame()

,0
Unnamed: 0,NCGC00178831-03
AW,54367203.0
AWeight,13.053
Arto,2.176
BertzCT,3.194
...,...
WPSA1,290.078
WPSA2,2301.941
WPSA3,59.492
grav,88.147
